# Bioblitz Analysis 2024

In [1]:
import pandas as pd
import numpy as np
import os
print(os.getcwd())
os.chdir("..")
import pandas as pd
import numpy as np
import folium
from folium import plugins
import matplotlib.colors as mcolors

c:\Users\ares\Desktop\Andrea\Leiden\Leiden University\Semestre 3\Statistical Consulting\Assignment\Submission code\Bioblitz


In [2]:
plants=pd.read_csv('Data/Bioblitz data/plants_wageningen_campus.csv')
plants.head()

,observation_id,observation_date,observation_time,species_name,species_name_scientific,species_group_id,species_group_name,life_stage_name,activity_name,method_name,location,lat,lng,month,day_of_year,week,lat_grid,lng_grid,cluster,distance_from_center
0,307345463,2024-05-02,08:15,Echte koekoeksbloem,Silene flos-cuculi,10,Plants,bloeiend,ter plaatse,onbekend,Wageningen - Campus,51.98819,5.66625,5,123,18,15,12,0,343.387427
1,307345475,2024-05-02,08:13,Pinksterbloem,Cardamine pratensis,10,Plants,onbekend,ter plaatse,onbekend,Wageningen - Campus,51.98807,5.66658,5,123,18,15,12,0,343.034156
2,307345478,2024-05-02,08:13,Scherpe boterbloem,Ranunculus acris,10,Plants,onbekend,ter plaatse,onbekend,Wageningen - Campus,51.98805,5.66663,5,123,18,15,12,0,342.985186
3,307345454,2024-05-02,08:16,Krulzuring,Rumex crispus,10,Plants,bloeiend,ter plaatse,onbekend,Wageningen - Campus,51.98820,5.66617,5,123,18,15,12,0,341.879388
4,307345473,2024-05-02,08:13,Gewoon reukgras,Anthoxanthum odoratum,10,Plants,onbekend,ter plaatse,onbekend,Wageningen - Campus,51.98807,5.66655,5,123,18,15,12,0,341.952578


In [3]:

dfv = plants.copy()
dfv["lat"] = pd.to_numeric(dfv["lat"], errors="coerce")
dfv["lng"] = pd.to_numeric(dfv["lng"], errors="coerce")
dfv = dfv.replace([np.inf, -np.inf], np.nan).dropna(subset=["lat", "lng"])
if dfv.empty:
    raise ValueError("Nessun punto valido (lat/lng mancanti).")

# 2) Centro mappa
center = [float(dfv["lat"].mean()), float(dfv["lng"].mean())]

# 3) Crea mappa con più basemap selezionabili
m = folium.Map(location=center, zoom_start=15, tiles=None)
folium.TileLayer("CartoDB positron", name="Chiara (Positron)").add_to(m)
folium.TileLayer("CartoDB dark_matter", name="Scura (Dark Matter)").add_to(m)
folium.TileLayer("OpenStreetMap", name="OpenStreetMap").add_to(m)
folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Esri",
    name="Satellite (Esri)"
).add_to(m)

# 4) Layer PUNTI (con popup utili)
points_fg = folium.FeatureGroup(name="Osservazioni (punti)", show=True)
for _, r in dfv.iterrows():
    popup_txt = (
        f"<b>{r.get('species_name','')}</b>"
        f"<br><i>{r.get('species_name_scientific','')}</i>"
        f"<br>Data: {r.get('observation_date','')}"
        f"<br>Ora: {r.get('observation_time','')}"
        f"<br>Luogo: {r.get('location','')}"
    )
    folium.CircleMarker(
        location=[float(r["lat"]), float(r["lng"])],
        radius=5,           # aumenta se vuoi punti più grandi
        color="#1f77b4",
        fill=True,
        fill_opacity=0.7,
        popup=popup_txt
    ).add_to(points_fg)
points_fg.add_to(m)

# 5) HEATMAP (densità osservazioni)
# Se vuoi pesare la heatmap con un campo (es. distanza, abbondanza), metti weights=... qui sotto.
heat_data = dfv[["lat", "lng"]].astype(float).values.tolist()
plugins.HeatMap(heat_data, radius=25, blur=20, max_zoom=17, name="HeatMap (densità)", show=False).add_to(m)

# 6) (Opzionale) HeatMap pesata da un campo, es. distance_from_center
# weights = dfv["distance_from_center"].astype(float).to_numpy()
# heat_w = [[la, lo, w] for (la, lo), w in zip(dfv[["lat","lng"]].to_numpy(float), weights)]
# plugins.HeatMap(heat_w, radius=25, blur=20, max_zoom=17, name="HeatMap (pesata)", show=False).add_to(m)

# 7) Controllo layer + salva
folium.LayerControl().add_to(m)
m.save("mappa_osservazioni.html")
print("Mappa salvata come mappa_osservazioni.html")


Mappa salvata come mappa_osservazioni.html


In [4]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import folium
from folium import plugins

# --- 1️⃣ Prepara i dati ---
df = plants.copy()  # o il tuo dataset
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=["lat", "lng"])

# --- 2️⃣ Seleziona le variabili per il clustering (solo coordinate) ---
X = df[["lat", "lng"]].to_numpy()

# --- 3️⃣ Applica KMeans con 104 cluster ---
n_clusters = 104
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
df["cluster_id"] = kmeans.fit_predict(X)

# --- 4️⃣ Calcola i centroidi ---
centroids = kmeans.cluster_centers_

# --- 5️⃣ Crea la mappa interattiva ---
center = [df["lat"].mean(), df["lng"].mean()]
# 3) Crea mappa con più basemap selezionabili
m = folium.Map(location=center, zoom_start=15, tiles=None)
folium.TileLayer("CartoDB positron", name="Chiara (Positron)").add_to(m)
folium.TileLayer("CartoDB dark_matter", name="Scura (Dark Matter)").add_to(m)
folium.TileLayer("OpenStreetMap", name="OpenStreetMap").add_to(m)
folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Esri",
    name="Satellite (Esri)"
).add_to(m)

# Colori diversi per ogni cluster (ricicla la palette)
import matplotlib.cm as cm
import matplotlib.colors as mcolors
palette = cm.get_cmap("tab20", n_clusters)

def cluster_color(cid):
    rgb = palette(cid % n_clusters)[:3]
    return mcolors.to_hex(rgb)

# Aggiungi punti
for _, r in df.iterrows():
    folium.CircleMarker(
        location=[r["lat"], r["lng"]],
        radius=4,
        color=cluster_color(int(r["cluster_id"])),
        fill=True,
        fill_opacity=0.8,
        popup=f"{r['species_name']}<br>Cluster: {r['cluster_id']}"
    ).add_to(m)

# Aggiungi centroidi
for i, (latc, lngc) in enumerate(centroids):
    folium.Marker(
        location=[latc, lngc],
        icon=folium.DivIcon(html=f"<b style='color:red;'>{i}</b>"),
        tooltip=f"Cluster {i}"
    ).add_to(m)

# --- 6️⃣ Heatmap per densità ---
plugins.HeatMap(df[["lat", "lng"]].values, radius=25, blur=20, name="HeatMap").add_to(m)

# --- 7️⃣ Controllo e salvataggio ---
folium.LayerControl().add_to(m)
m.save("cluster_map_104.html")
print("✅ Mappa salvata come cluster_map_104.html")


C:\Users\ares\AppData\Local\Temp\ipykernel_41240\1302386877.py:38: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  palette = cm.get_cmap("tab20", n_clusters)


✅ Mappa salvata come cluster_map_104.html


In [5]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# --- 1️⃣ Prepara il dataset ---
df = plants.copy()  # usa il tuo dataframe con colonne lat, lng
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=["lat", "lng"])

# --- 2️⃣ Clustering con KMeans ---
n_clusters = 104
X = df[["lat", "lng"]].to_numpy()
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
df["cluster_id"] = kmeans.fit_predict(X)

# --- 3️⃣ Calcolo centroidi e dimensione di ogni cluster ---
cluster_summary = (
    df.groupby("cluster_id")
      .agg(lat_center=("lat", "mean"),
           lng_center=("lng", "mean"),
           size=("cluster_id", "count"))
      .reset_index()
)

# --- 4️⃣ Crea la mappa ---
center = [df["lat"].mean(), df["lng"].mean()]
m = folium.Map(location=center, zoom_start=15, tiles=None)
folium.TileLayer("CartoDB positron", name="Chiara (Positron)").add_to(m)
folium.TileLayer("CartoDB dark_matter", name="Scura (Dark Matter)").add_to(m)
folium.TileLayer("OpenStreetMap", name="OpenStreetMap").add_to(m)
folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Esri",
    name="Satellite (Esri)"
).add_to(m)

# palette colori
palette = cm.get_cmap("tab20", n_clusters)
def cluster_color(cid):
    rgb = palette(cid % n_clusters)[:3]
    return mcolors.to_hex(rgb)

# --- 5️⃣ Aggiungi solo i centroidi (1 punto per cluster) ---
size_min, size_max = cluster_summary["size"].min(), cluster_summary["size"].max()

for _, r in cluster_summary.iterrows():
    # scala la dimensione del cerchio in base al numero di punti
    radius = 5 + 30 * (r["size"] - size_min) / (size_max - size_min) if size_max > size_min else 10
    folium.CircleMarker(
        location=[r["lat_center"], r["lng_center"]],
        radius=float(radius),
        color=cluster_color(int(r["cluster_id"])),
        fill=True,
        fill_color=cluster_color(int(r["cluster_id"])),
        fill_opacity=0.7,
        popup=(f"<b>Cluster {r['cluster_id']}</b><br>"
               f"Dimensione: {int(r['size'])} osservazioni<br>"
               f"Lat: {r['lat_center']:.5f}<br>"
               f"Lng: {r['lng_center']:.5f}")
    ).add_to(m)

# --- 6️⃣ Controllo e salvataggio ---
folium.LayerControl().add_to(m)
m.save("cluster_centroids_map.html")
print("✅ Mappa salvata come cluster_centroids_map.html")


✅ Mappa salvata come cluster_centroids_map.html


C:\Users\ares\AppData\Local\Temp\ipykernel_41240\3716096413.py:40: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  palette = cm.get_cmap("tab20", n_clusters)
